In [3]:
import pandas as pd
import json

# Tecniche di ML a confronto
In questo jupyter vengono confrontate le prestazioni delle varie classificazioni di ciascun modello.
Il metodo utilizzato è CountVectorizer con una tokenizzazione regex.

Per la realizzazione ho seguito https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

In [4]:
dataset = pd.read_csv('../description_predictions.csv', index_col=0)
with open("../sinonimi.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [5]:
dataset_withoutNAN = dataset[dataset.prediction != 'NAN']

dataset_withoutNAN.reset_index(drop=True, inplace=True)

Il codice seguente serve per convertire i soprannomi delle squadre in home team e away team

In [6]:
"""for i, row in dataset_withoutNAN.iterrows():
    h_team, a_team, description, prediction = row.h_team, row.a_team, row.description, row.prediction

    syn = {}
    #cerco nel dizionario di sinonimi, tutti i sinonimi delle squadre del match
    for key in jsonObject.keys():
        if (h_team in key) or (key in h_team):
            syn['home team'] = jsonObject[key] 
            
        if (a_team in key) or (key in a_team):
            syn['away team'] = jsonObject[key] 

    #successivamente prendo il testo e sostituisco i sinonimi con home o away team
    description = description.lower()
    for key in syn.keys():
        for val in syn[key]:
            description = description.replace(val.lower(), key)

    dataset_withoutNAN.at[i, 'description'] = description
"""

"for i, row in dataset_withoutNAN.iterrows():\n    h_team, a_team, description, prediction = row.h_team, row.a_team, row.description, row.prediction\n\n    syn = {}\n    #cerco nel dizionario di sinonimi, tutti i sinonimi delle squadre del match\n    for key in jsonObject.keys():\n        if (h_team in key) or (key in h_team):\n            syn['home team'] = jsonObject[key] \n            \n        if (a_team in key) or (key in a_team):\n            syn['away team'] = jsonObject[key] \n\n    #successivamente prendo il testo e sostituisco i sinonimi con home o away team\n    description = description.lower()\n    for key in syn.keys():\n        for val in syn[key]:\n            description = description.replace(val.lower(), key)\n\n    dataset_withoutNAN.at[i, 'description'] = description\n"

In [7]:
dataset_withoutNAN = dataset_withoutNAN[['description', 'prediction']]
prediction_labels = {
    'N': 0,
    'V': 1,
    'P': 2
}

dataset_withoutNAN['prediction'] = dataset_withoutNAN['prediction'].map(prediction_labels)
dataset_withoutNAN.head()

,description,prediction
0,Verona and Sassuolo take on each other in a hi...,0
1,Inter begin their title defence with a match a...,1
2,Atalanta will look to make a flying start to t...,2
3,Newcomers Empoli will be aiming to kick off th...,2
4,After failing to impress in the 2020/21 Serie ...,2


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True, stop_words='english', ngram_range = (2,2), tokenizer = token.tokenize)
text_counts= cv.fit_transform(dataset_withoutNAN.description)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, dataset_withoutNAN.prediction, test_size=0.3, random_state=1)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as mtr


models = {
    'Logistic Regression': LogisticRegression(max_iter=10000), #max_iter di default vale 100, ho dovuto alzarlo se no non converge
    'Support Vector Machine': SVC(),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

In [11]:
predictions = {}
for model_name, model in models.items():
    models[model_name].fit(X_train, y_train)
    predictions[model_name] = model.predict(X_test)

E = []
for estimator, y_pred in predictions.items():
    report = mtr.classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    E.append({
        'Model': estimator, 'Accuracy': report['accuracy'],
        'Avg Precision (macro)': report['macro avg']['precision'],
        'Avg Recall (macro)': report['macro avg']['recall'],
        'Avg F1-score (macro)': report['macro avg']['f1-score'],
        'Avg Precision (weighted)': report['weighted avg']['precision'],
        'Avg Recall (weighted)': report['weighted avg']['recall'],
        'Avg F1-score (weighted)': report['weighted avg']['f1-score']
    })

E = pd.DataFrame(E).set_index('Model', inplace=False)

In [12]:
E

,Accuracy,Avg Precision (macro),Avg Recall (macro),Avg F1-score (macro),Avg Precision (weighted),Avg Recall (weighted),Avg F1-score (weighted)
Model,,,,,,,
Logistic Regression,0.657895,0.689363,0.580004,0.572764,0.678354,0.657895,0.612296
Support Vector Machine,0.464912,0.486726,0.343434,0.229709,0.499379,0.464912,0.304535
Multinomial Naive Bayes,0.657895,0.619405,0.608626,0.595898,0.634117,0.657895,0.630704
Decision Tree,0.754386,0.764792,0.716120,0.731162,0.765504,0.754386,0.750521
Random Forest,0.587719,0.821315,0.486242,0.447778,0.766595,0.587719,0.502076
K-Nearest Neighbors,0.500000,0.460883,0.458142,0.428231,0.477247,0.500000,0.463540
